In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
def csv(file):
    dat_ = pd.read_csv(file + '.csv', encoding = 'cp949')
    return dat_
    
def ex(file):
    dat_ = pd.read_excel(file + '.xlsx', header = 1)
    return dat_

path = 'data/'

# 승객 유형, 노인 21, 22 옆으로 합치기

In [45]:
승객유형별21 = csv(path + '서울교통공사_1-8호선 역별 일별 시간대별 승객유형별 승하차인원_20211231')
승객유형별22 = csv(path + '서울교통공사_1_8호선 역별 일별 시간대별 승객유형별 승하차인원_20220531')

노인21 = csv(path+'서울교통공사_역별 일별 시간대별 노인 승하차인원_20211231')
노인22 = csv(path+'서울교통공사_역별 일별 시간대별 노인 승하차인원 정보_20220531')

In [7]:
data = pd.read_csv('data 시간대별 승하차 승객 수 (날짜 역명으로 정렬).csv')

## 노인 전처리
1. 컬럼명 일자 -> 날짜로 바꿔준다.
2. 노인22의 구분의 value를 승차, 하차로 바꿔준다.
3. 호선열이 없어서 합칠 때 문제가 발생할 수 있으므로 채워준다.

In [33]:
# 서울호선역명

In [47]:
# 노인21.shape[0]
+ 노인22.shape[0]

노인21.rename(columns = {'일자':'날짜'}, inplace = True)
노인22.rename(columns = {'일자':'날짜'}, inplace = True)

노인22.구분 = 노인22.구분.map({'합계 : 승차':'승차', '합계 : 하차':'하차'})


### 노인에 호선 추가
# data.sort_values(by = ['호선','역명'])
서울호선역명 = list(data[['호선','역번호']].groupby('호선').value_counts().index) # .sort_values('역명')
# 서울호선역명
서울호선역명dic = {i[1]:i[0] for i in 서울호선역명} # 역번호를 호선으로 mapping할 것임.

노인21['호선'] = 노인21['역번호'].map(서울호선역명dic)
노인22['호선'] = 노인22['역번호'].map(서울호선역명dic)

## 시간대 column명 변경
시간대 = ['06시이전', '06시-07시', '07시-08시',
       '08시-09시', '09시-10시', '10시-11시', '11시-12시', '12시-13시', '13시-14시',
       '14시-15시', '15시-16시', '16시-17시', '17시-18시', '18시-19시', '19시-20시',
       '20시-21시', '21시-22시', '22시-23시', '23시이후']
노인21.rename(columns = {i:f'노인_{i}' for i in 시간대}, inplace = True)
노인22.rename(columns = {i:f'노인_{i}' for i in 시간대}, inplace = True)

## columns 순서 변경
노인21 = 노인21[['연번', '날짜', '호선', '역번호', '역명', '구분', '노인_06시이전', '노인_06시-07시', '노인_07시-08시',
       '노인_08시-09시', '노인_09시-10시', '노인_10시-11시', '노인_11시-12시', '노인_12시-13시',
       '노인_13시-14시', '노인_14시-15시', '노인_15시-16시', '노인_16시-17시', '노인_17시-18시',
       '노인_18시-19시', '노인_19시-20시', '노인_20시-21시', '노인_21시-22시', '노인_22시-23시',
       '노인_23시이후']]

노인22 = 노인22[노인21.columns]

In [58]:
노인 = pd.concat([노인21,노인22], axis=0)
노인 = 노인.sort_values(by=['날짜', '역명']).reset_index(drop=True)

In [59]:
# 노인.to_csv('노인 시간대별 승하차 승객 수 (날짜 역명으로 정렬).csv')

# 승객 유형별로 다 끊어서 따로따로 저장

In [71]:
승객유형별21= 승객유형별21.sort_values(by=['날짜', '역명']).reset_index(drop=True)
승객유형별22 = 승객유형별22.sort_values(by=['날짜', '역명']).reset_index(drop=True)

In [72]:
for i in 승객유형별21['승객유형'].unique():
    globals()[f'{i}21'] = 승객유형별21[승객유형별21['승객유형'] == i]
    globals()[f'{i}22'] = 승객유형별22[승객유형별22['승객유형'] == i]

In [73]:
## 시간대 column명 변경
시간대 = ['06시이전', '06시-07시', '07시-08시',
       '08시-09시', '09시-10시', '10시-11시', '11시-12시', '12시-13시', '13시-14시',
       '14시-15시', '15시-16시', '16시-17시', '17시-18시', '18시-19시', '19시-20시',
       '20시-21시', '21시-22시', '22시-23시', '23시이후']

for i in 승객유형별21['승객유형'].unique():
    globals()[f'{i}21'].rename(columns = {j:f'{i}_{j}' for j in 시간대}, inplace = True)
    globals()[f'{i}22'].rename(columns = {j:f'{i}_{j}' for j in 시간대}, inplace = True)

In [ ]:
for i in 승객유형별21['승객유형'].unique():
    globals()[f'{i}21'].head()
    globals()[f'{i}22'].head()
    break

In [100]:
# import datetime as dt
from datetime import datetime

jul01_21 = datetime.strptime('2021-07-01', "%Y-%m-%d").date()
dec31_21 = datetime.strptime('2021-12-31', "%Y-%m-%d").date()

(dec31_21 - jul01_21).days

jan01_22 = datetime.strptime('2022-01-01', "%Y-%m-%d").date()
# jan01_22 - dec31_21 # 하루
may31_22 = datetime.strptime('2022-05-31', "%Y-%m-%d").date()

(may31_22 - jan01_22).days

183

150

In [107]:
승객유형별21.역번호.nunique()
승객유형별22.역번호.nunique()
data.역번호.nunique()
노인22.역번호.nunique()

284

283

284

280